# 확산 모델 학습

무조건적인 이미지 생성은 학습에 사용된 데이터셋의 이미지와 유사한 이미지를 생성하는 확산 모델의 인기 있는 응용 프로그램입니다. 일반적으로 가장 좋은 결과는 특정 데이터셋에 사전 학습된 모델을 미세 조정하여 얻습니다. 이러한 체크포인트는 [Hub](https://huggingface.co/search/full-text?q=unconditional-image-generation&type=model)에서 많이 찾을 수 있지만, 마음에 드는 것을 찾을 수 없다면 언제든지 직접 학습할 수 있습니다!

이 튜토리얼에서는 [Smithsonian Butterflies](https://huggingface.co/datasets/huggan/smithsonian_butterflies_subset) 데이터셋의 하위 집합에서 [UNet2DModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d#diffusers.UNet2DModel)을 처음부터 학습하여 자신만의 🦋 나비 🦋를 생성하는 방법을 알려드립니다.

<Tip>

💡 이 학습 튜토리얼은 [Training with 🧨 Diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/training_example.ipynb) 노트북을 기반으로 합니다. 확산 모델의 작동 방식과 같은 추가 세부 정보 및 컨텍스트는 노트북을 확인하세요!

</Tip>

시작하기 전에 이미지 데이터셋을 로드하고 전처리하기 위한 🤗 Datasets와 모든 수의 GPU에서 학습을 단순화하기 위한 🤗 Accelerate가 설치되어 있는지 확인하세요. 다음 명령은 학습 메트릭을 시각화하기 위한 [TensorBoard](https://www.tensorflow.org/tensorboard)도 설치합니다(학습을 추적하기 위해 [Weights & Biases](https://docs.wandb.ai/)를 사용할 수도 있습니다).

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석을 해제하세요.
#!pip install diffusers[training]

커뮤니티와 모델을 공유하는 것을 권장하며, 그렇게 하려면 Hugging Face 계정에 로그인해야 합니다(아직 계정이 없다면 [여기](https://hf.co/join)에서 만드세요!). 노트북에서 로그인하고 메시지가 표시되면 토큰을 입력할 수 있습니다. 토큰에 쓰기 역할이 있는지 확인하세요.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

또는 터미널에서 로그인합니다:

```bash
huggingface-cli login
```

모델 체크포인트가 상당히 크므로 이러한 대용량 파일을 버전 관리하기 위해 [Git-LFS](https://git-lfs.com/)를 설치합니다:

```bash
!sudo apt -qq install git-lfs
!git config --global credential.helper store
```

## 학습 구성

편의를 위해 학습 하이퍼파라미터를 포함하는 `TrainingConfig` 클래스를 만듭니다(자유롭게 조정하세요):

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # 생성된 이미지 해상도
    train_batch_size = 16
    eval_batch_size = 16  # 평가 중 샘플링할 이미지 수
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"  # float32의 경우 `no`, 자동 혼합 정밀도의 경우 `fp16`
    output_dir = "ddpm-butterflies-128"  # 로컬 및 HF Hub의 모델 이름

    push_to_hub = True  # 저장된 모델을 HF Hub에 업로드할지 여부
    hub_model_id = "<your-username>/<my-awesome-model>"  # HF Hub에 만들 리포지토리 이름
    hub_private_repo = None
    overwrite_output_dir = True  # 노트북을 다시 실행할 때 이전 모델을 덮어쓸지 여부
    seed = 0


config = TrainingConfig()

## 데이터셋 로드

🤗 Datasets 라이브러리를 사용하면 [Smithsonian Butterflies](https://huggingface.co/datasets/huggan/smithsonian_butterflies_subset) 데이터셋을 쉽게 로드할 수 있습니다.

In [ ]:
from datasets import load_dataset

config.dataset_name = "huggan/smithsonian_butterflies_subset"
dataset = load_dataset(config.dataset_name, split="train")

<Tip>

💡 추가 데이터셋은 [HugGan Community Event](https://huggingface.co/huggan)에서 찾거나 로컬 [`ImageFolder`](https://huggingface.co/docs/datasets/image_dataset#imagefolder)를 만들어 자체 데이터셋을 사용할 수 있습니다. HugGan Community Event의 데이터셋인 경우 `config.dataset_name`을 데이터셋의 리포지토리 ID로 설정하거나 자체 이미지를 사용하는 경우 `imagefolder`로 설정합니다.

</Tip>

🤗 Datasets는 [Image](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Image) 기능을 사용하여 이미지 데이터를 자동으로 디코딩하고 시각화할 수 있는 [`PIL.Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html)로 로드합니다.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for i, image in enumerate(dataset[:4]["image"]):
    axs[i].imshow(image)
    axs[i].set_axis_off()
fig.show()

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/butterflies_ds.png"/>
</div>

하지만 이미지 크기가 모두 다르므로 먼저 전처리해야 합니다.

* `Resize`는 이미지 크기를 `config.image_size`에 정의된 크기로 변경합니다.
* `RandomHorizontalFlip`은 이미지를 무작위로 미러링하여 데이터셋을 증강합니다.
* `Normalize`는 픽셀 값을 모델이 예상하는 [-1, 1] 범위로 다시 조정하는 데 중요합니다.

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

🤗 Datasets의 [set_transform](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.set_transform) 메서드를 사용하여 학습 중에 즉시 `preprocess` 함수를 적용합니다.

In [ ]:
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}


dataset.set_transform(transform)

이미지 크기가 조정되었는지 확인하기 위해 이미지를 다시 시각화해 보세요. 이제 학습을 위해 데이터셋을 [DataLoader](https://pytorch.org/docs/stable/data#torch.utils.data.DataLoader)로 래핑할 준비가 되었습니다!

In [ ]:
import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

## UNet2DModel 생성

🧨 Diffusers의 사전 학습된 모델은 원하는 매개변수를 사용하여 모델 클래스에서 쉽게 만들 수 있습니다. 예를 들어 [UNet2DModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d#diffusers.UNet2DModel)을 만들려면 다음을 수행합니다.

In [ ]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # 대상 이미지 해상도
    in_channels=3,  # 입력 채널 수, RGB 이미지의 경우 3
    out_channels=3,  # 출력 채널 수
    layers_per_block=2,  # UNet 블록당 사용할 ResNet 레이어 수
    block_out_channels=(128, 128, 256, 256, 512, 512),  # 각 UNet 블록의 출력 채널 수
    down_block_types=(
        "DownBlock2D",  # 일반 ResNet 다운샘플링 블록
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # 공간적 자기 주의 기능이 있는 ResNet 다운샘플링 블록
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # 일반 ResNet 업샘플링 블록
        "AttnUpBlock2D",  # 공간적 자기 주의 기능이 있는 ResNet 업샘플링 블록
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

샘플 이미지 모양이 모델 출력 모양과 일치하는지 빠르게 확인하는 것이 좋습니다.

In [ ]:
sample_image = dataset[0]["images"].unsqueeze(0)
print("Input shape:", sample_image.shape)

Input shape: torch.Size([1, 3, 128, 128])

In [ ]:
print("Output shape:", model(sample_image, timestep=0).sample.shape)

Output shape: torch.Size([1, 3, 128, 128])

훌륭합니다! 다음으로 이미지에 약간의 노이즈를 추가할 스케줄러가 필요합니다.

## 스케줄러 생성

스케줄러는 모델을 학습용으로 사용하는지 추론용으로 사용하는지에 따라 다르게 동작합니다. 추론 중에는 스케줄러가 노이즈에서 이미지를 생성합니다. 학습 중에는 스케줄러가 확산 프로세스의 특정 지점에서 모델 출력(또는 샘플)을 가져와 *노이즈 스케줄* 및 *업데이트 규칙*에 따라 이미지에 노이즈를 적용합니다.

[DDPMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddpm#diffusers.DDPMScheduler)를 살펴보고 `add_noise` 메서드를 사용하여 이전 `sample_image`에 임의의 노이즈를 추가해 보겠습니다.

In [ ]:
import torch
from PIL import Image
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
noise = torch.randn(sample_image.shape)
timesteps = torch.LongTensor([50])
noisy_image = noise_scheduler.add_noise(sample_image, noise, timesteps)

Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/noisy_butterfly.png"/>
</div>

모델의 학습 목표는 이미지에 추가된 노이즈를 예측하는 것입니다. 이 단계의 손실은 다음과 같이 계산할 수 있습니다.

In [ ]:
import torch.nn.functional as F

noise_pred = model(noisy_image, timesteps).sample
loss = F.mse_loss(noise_pred, noise)

## 모델 학습

이제 모델 학습을 시작하는 데 필요한 대부분의 구성 요소를 갖추었으며, 남은 것은 모든 것을 통합하는 것뿐입니다.

먼저 최적화 프로그램과 학습률 스케줄러가 필요합니다.

In [ ]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

다음으로 모델을 평가할 방법이 필요합니다. 평가를 위해 [DDPMPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/ddpm#diffusers.DDPMPipeline)을 사용하여 샘플 이미지 배치를 생성하고 그리드로 저장할 수 있습니다.

In [ ]:
from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os

def evaluate(config, epoch, pipeline):
    # 임의의 노이즈에서 일부 이미지를 샘플링합니다(역방향 확산 프로세스).
    # 기본 파이프라인 출력 유형은 `List[PIL.Image]`입니다.
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed), # 주 학습 루프의 임의 상태를 되감지 않도록 별도의 토치 생성기를 사용합니다.
    ).images

    # 이미지에서 그리드를 만듭니다.
    image_grid = make_image_grid(images, rows=4, cols=4)

    # 이미지를 저장합니다.
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

이제 이러한 모든 구성 요소를 TensorBoard 로깅, 그래디언트 누적 및 혼합 정밀도 학습을 위해 🤗 Accelerate를 사용하여 학습 루프에 함께 래핑할 수 있습니다. 모델을 Hub에 업로드하려면 리포지토리 이름과 정보를 가져온 다음 Hub에 푸시하는 함수를 작성합니다.

<Tip>

💡 아래의 학습 루프는 위협적이고 길어 보일 수 있지만, 나중에 단 한 줄의 코드로 학습을 시작할 때 그만한 가치가 있을 것입니다! 기다릴 수 없고 이미지 생성을 시작하고 싶다면 아래 코드를 복사하여 실행하세요. 언제든지 나중에 돌아와서 학습 루프를 더 자세히 살펴볼 수 있습니다(예: 모델 학습이 완료되기를 기다리는 동안). 🤗

</Tip>

In [ ]:
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # 가속기 및 텐서보드 로깅 초기화
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # 모든 것을 준비합니다.
    # 기억해야 할 특정 순서는 없으며, 준비 메서드에 전달한 것과 동일한 순서로 객체의 압축을 풀기만 하면 됩니다.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # 이제 모델을 학습합니다.
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]
            # 이미지에 추가할 노이즈 샘플링
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # 각 이미지에 대한 임의의 시간 단계 샘플링
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
                dtype=torch.int64
            )

            # 각 시간 단계의 노이즈 크기에 따라 깨끗한 이미지에 노이즈 추가
            # (정방향 확산 프로세스)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # 노이즈 잔차 예측
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # 각 에포크 후 선택적으로 evaluate()를 사용하여 일부 데모 이미지를 샘플링하고 모델을 저장합니다.
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    upload_folder(
                        repo_id=repo_id,
                        folder_path=config.output_dir,
                        commit_message=f"Epoch {epoch}",
                        ignore_patterns=["step_*", "epoch_*"],
                    )
                else:
                    pipeline.save_pretrained(config.output_dir)

휴, 코드가 꽤 많았죠! 하지만 마침내 🤗 Accelerate의 [notebook_launcher](https://huggingface.co/docs/accelerate/main/en/package_reference/launchers#accelerate.notebook_launcher) 함수로 학습을 시작할 준비가 되었습니다. 함수에 학습 루프, 모든 학습 인수, 학습에 사용할 프로세스 수(이 값을 사용 가능한 GPU 수로 변경할 수 있음)를 전달합니다.

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

학습이 완료되면 확산 모델에서 생성된 최종 🦋 이미지 🦋를 살펴보세요!

In [ ]:
import glob

sample_images = sorted(glob.glob(f"{config.output_dir}/samples/*.png"))
Image.open(sample_images[-1])

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/butterflies_final.png"/>
</div>

## 다음 단계

무조건적인 이미지 생성은 학습할 수 있는 작업의 한 예입니다. [🧨 Diffusers 학습 예제](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/overview) 페이지를 방문하여 다른 작업과 학습 기술을 탐색할 수 있습니다. 다음은 학습할 수 있는 몇 가지 예입니다.

* [Textual Inversion](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/text_inversion), 특정 시각적 개념을 모델에 가르치고 생성된 이미지에 통합하는 알고리즘입니다.
* [DreamBooth](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/dreambooth), 피사체의 여러 입력 이미지가 주어졌을 때 피사체의 개인화된 이미지를 생성하는 기술입니다.
* [가이드](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/text2image) 자체 데이터셋에서 Stable Diffusion 모델을 미세 조정합니다.
* [가이드](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/lora) LoRA를 사용하여 매우 큰 모델을 더 빠르게 미세 조정하는 메모리 효율적인 기술입니다.